In [3]:
import os


os.path.abspath('')

'c:\\Users\\Hannah Blaurock\\Desktop\\AnomalyDetect\\AnDetect'

In [2]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, WeightedRandomSampler

import Process_data
from Model_TestTrain import MLP_Residual, train_model, test_model

##GET THE DATA
gen_data = False #do we want to use generated data for the sanity check
use_gaussian_data = False #Gaussian data currently not useable (minor fixes needed)


if gen_data:
    test_df, train_ds, test_y = Process_data.use_gen_data(
        mean_var_error = (1, 0.5, 1.5, 1), #(mu, sigma of mean dist; mu, sigma of var dist if there is an error)
        mean_var_noError = (1, 1, 1, 0.5) #(mu, sigma of mean dist; mu, sigma of var dist if there is NO error)
    )
else:
    df = Process_data.get_data()
    train_ds, test_ds, test_y = Process_data.data_preprocess(df)


Dataset: All Data
Partition: 100.00%
Total Length: 37824
Errors: Ussing Error-Threshold 0
Number of Errors: 0.054621404399323184
Number of Non-errors: 0.9453785956006768
Percent of non-errors: 0.00%

Dataset: Trainingdata
Partition: 65.01%
Total Length: 24589
Errors: Ussing Error-Threshold 0
Number of Errors: 0.07222741876448818
Number of Non-errors: 0.9277725812355118
Percent of non-errors: 0.00%

Dataset: Testdata
Partition: 34.99%
Total Length: 13235
Errors: Ussing Error-Threshold 0
Number of Errors: 0.0219115980355119
Number of Non-errors: 0.9780884019644881
Percent of non-errors: 0.01%



In [6]:
train_ds.__getitem__(1)[0].shape[0]

256